# TP1 HomeWork

In [2]:
import numpy as np
from main import *

tps écoulé (gradient_rho_constant): 0.015623092651367188
[-0.69056385  0.15469913 -0.61430107  0.4909626  -0.04630803]


 **We define an adaptative version of the gradient descend algorithm** 

In [3]:
def gradient_rho_adaptatif(fun, fun_der, U0, rho, tol,args):
    
    # Fonction permettant de minimiser la fonction f(U) par rapport au vecteur U 
    # Méthode : gradient à pas fixe
    # INPUTS :
    # - han_f   : handle vers la fonction à minimiser
    # - han_df  : handle vers le gradient de la fonction à minimiser
    # - U0      : vecteur initial 
    # - rho     : paramètre gérant l'amplitude des déplacement 
    # - tol     : tolérance pour définir le critère d'arrêt
    # OUTPUT : 
    # - GradResults : structure décrivant la solution


    itermax=10000  # nombre maximal d'itérations 
    xn=U0
    f=fun(xn,*args) # point initial de l'algorithme
    it=0         # compteur pour les itérations
    converged = False;
    
    while (~converged & (it < itermax)):
        it=it+1
        dfx=fun_der(xn,*args)# valeur courante de la fonction à minimiser
        
        xnp1=xn-rho*dfx
        fnp1 = fun(xnp1,*args)
        
        if fnp1 < f :
            
            if abs(fnp1-f)<tol:
                converged = True
                
            xn, f = xnp1, fnp1
            rho *= 2
        else :
            rho /= 2

    GradResults = {
            'initial_x':U0,
            'minimum':xnp1,
            'f_minimum':fnp1,
            'iterations':it,
            'converged':converged
            }
    return GradResults

In [4]:
gradient_rho_adaptatif(f1,df1,x0,rho=1,tol=1e-6,args=(B,S))

{'initial_x': array([1., 1., 1., 1., 1.]),
 'minimum': array([-0.67000244,  0.1425606 , -0.61584071,  0.47887395, -0.02041732]),
 'f_minimum': -1.8361443391078156,
 'iterations': 386,
 'converged': True}

## On va plot le nombre d'itération par rapport à rho

## Méthodes Quasi-Newton

In [5]:
from scipy.optimize import minimize

In [6]:
minimize(fun = f1, x0 = x0, args=(B,S), method='BFGS', tol=1e-6)

      fun: -1.836962311965236
 hess_inv: array([[ 0.50041674, -0.21238233,  0.05674326, -0.28046232,  0.50962222],
       [-0.21238233,  0.22402876,  0.03714748,  0.11546007, -0.34089541],
       [ 0.05674326,  0.03714748,  0.12473992, -0.04895957, -0.09129864],
       [-0.28046232,  0.11546007, -0.04895957,  0.20494128, -0.28975672],
       [ 0.50962222, -0.34089541, -0.09129864, -0.28975672,  0.77797771]])
      jac: array([ 0.00000000e+00,  8.94069672e-08, -2.98023224e-08,  1.49011612e-08,
        1.49011612e-08])
  message: 'Optimization terminated successfully.'
     nfev: 91
      nit: 10
     njev: 13
   status: 0
  success: True
        x: array([-0.6960314 ,  0.15793136, -0.61407083,  0.49414156, -0.05345805])

## Résolution analytique

In [7]:
np.linalg.eigvals(S)

array([45.48200751, 14.49627115,  0.34399006,  1.89569112,  7.21954015])

solution analytique existe

In [8]:
S_1 = np.linalg.inv(S)
solution = 0.5*S_1*B
np.abs(f1(solution, B, S) - minimize(fun = f1, x0 = x0, args=(B,S), method='BFGS', tol=1e-6)['fun'])

1.6653345369377348e-14

## Optimisation sous contraintes

In [9]:
minimize(fun = f1, x0 = x0, args=(B,S), method='SLSQP', bounds=[(0,1)]*5, tol=1e-6)

     fun: -0.13853161426486305
     jac: array([5.65140147e-01, 1.70477480e-03, 4.86906105e+00, 1.77648850e-03,
       2.70384207e-01])
 message: 'Optimization terminated successfully.'
    nfev: 65
     nit: 9
    njev: 9
  status: 0
 success: True
       x: array([0.00000000e+00, 1.26984956e-01, 0.00000000e+00, 1.94536138e-02,
       2.16628331e-15])

In [10]:
def f2(U,S):
    n=U.shape[0]
    U=np.matrix(U)
    U.shape=(n,1)
    fU = np.transpose(U) * S * U + np.transpose(U) * np.exp(U);
    return float(fU)

In [11]:
minimize(fun = f2, x0 = x0, args=(S), method='SLSQP', bounds=[(0,1)]*5, tol=1e-6)

     fun: 2.976067656614975e-15
     jac: array([1.00000018, 1.00000013, 1.00000035, 1.00000023, 1.00000022])
 message: 'Optimization terminated successfully.'
    nfev: 28
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([1.72929872e-16, 2.17964615e-15, 5.95736056e-16, 0.00000000e+00,
       2.77555756e-17])

## Optimisation sous contraintes et pénalisation

In [12]:
#fonction de pénalisation classique 
Beta = lambda u : np.sum(np.maximum(u-1, 0)**2 + (np.maximum(-u,0))**2)
print(Beta(x0), Beta(x0 - 2))

0.0 5.0


In [13]:
epsilon, start = 1/2, x0
for k in range(100):
    f1_penal = lambda U : f1(U,B,S) + (1/epsilon)*Beta(U)
    start = minimize(fun = f1_penal, x0 = start, method='BFGS', tol=1e-6)['x']
    epsilon /= 2
    
print(f1(start, B , S))
print(start)

-0.13853173459306484
[2.27345020e-09 1.26885930e-01 2.95427674e-10 1.94103684e-02
 9.25641783e-10]


In [14]:
epsilon, start = 1/2, x0
for k in range(1000):
    f2_penal = lambda U : f2(U,S) + (1/epsilon)*Beta(U)
    start = minimize(fun = f2_penal, x0 = start, method='BFGS', tol=1e-6)['x']
    epsilon /= 2
    
print(f2(start , S))
print(start)

8.152761402346261e-11
[7.19742181e-17 1.09058344e-11 1.98749448e-11 2.79335748e-11
 2.28131881e-11]


## Méthodes duales 

In [41]:
# Lagrangien implementation 

def L1(U,B,S,p) : 
    vec = []
    for i in U :
        vec.append(-i)
        vec.append(i-1)
    return f1(U,B,S) + np.sum(p*np.array(vec))

def dL1(U,B,S,p) : 
    vec = []
    for i in U :
        vec.append(-i)
        vec.append(i-1)
    return np.array(vec)
    


In [57]:
rho = 0.1 
p = np.random.normal(0,1,10)
for i in range(1000) : 
    x_min = minimize(fun = L1, x0 = x0, args=(B,S,p), method='BFGS', tol=1e-6)['x']
    p = np.maximum(0 , p + rho*dL1(x_min,B,S,p))

In [58]:
L1(x_min,B,S,p)

-0.1388585904269647

## Simulated Annealing 

In [79]:
def f3(U) : 
    return f1(U,B,S) + 10*np.sin(2*f1(U,B,S))
for i in range(5) : 
    x_init = np.random.uniform(0,1,5)
    print("for try {}/5 we get : {}".format(i+1,minimize(fun = f3, x0 = x_init, method='BFGS', tol=1e-6)['fun']))

for try 1/5 we get : 1.7684698448826008
for try 2/5 we get : 33.184396381016995
for try 3/5 we get : 33.184396381336214
for try 4/5 we get : 52.033952302469835
for try 5/5 we get : -1.3731228087495317


In [80]:
from scipy.optimize import basinhopping 


In [119]:
for i in range(5) : 
    x_init = np.random.uniform(0,1,5)
    print("for try {}/5 we get : {}".format(i+1,basinhopping(f3, x0 )['fun']))
    print("for try {}/5 we get : {}".format(i+1,basinhopping(f3, x0 )['x']))

for try 1/5 we get : -10.797900769519423
for try 1/5 we get : [ 0.02251051  0.12053079 -0.25249724  0.12418868  0.16967231]
for try 2/5 we get : -10.797900769519423
for try 2/5 we get : [-0.5485277   0.26902965 -0.860055    0.50548761  0.39285709]
for try 3/5 we get : -10.797900769519423
for try 3/5 we get : [-0.40728818  0.25642756 -0.68196585  0.57163556  0.1880288 ]
for try 4/5 we get : -10.797900769519423
for try 4/5 we get : [-1.41516198  0.53603595 -0.84470001  0.90836063 -0.80750152]
for try 5/5 we get : -10.797900769519423
for try 5/5 we get : [-0.28337391 -0.26499577 -0.46094914  0.29256113  0.18814096]


In [113]:
tf = lambda t: 0.8*t #temperature function
itf = lambda length: np.ceil(1.2*length) #iteration function
pfxs = lambda s, x: x + s*np.random.normal()
from functools import partial
pf = partial(pfxs, 0.1)

def sa(energyfunc, initials, epochs, tempfunc, iterfunc, proposalfunc):
    accumulator=[]
    best_solution = old_solution = initials['solution']
    T=initials['T']
    length=initials['length']
    best_energy = old_energy = energyfunc(old_solution)
    accepted=0
    total=0
    for index in range(epochs):
        print("Epoch", index)
        if index > 0:
            T = tempfunc(T)
            length=int(iterfunc(length))
        print("Temperature", T, "Length", length)
        for it in range(length):
            total+=1
            new_solution = proposalfunc(old_solution)
            new_energy = energyfunc(new_solution)
            # Use a min here as you could get a "probability" > 1
            alpha = min(1, np.exp((old_energy - new_energy)/T))
            if ((new_energy < old_energy) or (np.random.uniform() < alpha)):
                # Accept proposed solution
                accepted+=1
                accumulator.append((T, new_solution, new_energy))
                if new_energy < best_energy:
                    # Replace previous best with this one
                    best_energy = new_energy
                    best_solution = new_solution
                    best_index=total
                    best_temp=T
                old_energy = new_energy
                old_solution = new_solution
            else:
                # Keep the old stuff
                accumulator.append((T, old_solution, old_energy))
    
    best_meta=dict(index=best_index, temp=best_temp)
    print("frac accepted", accepted/total, "total iterations", total, 'bmeta', best_meta)
    return best_meta, best_solution, best_energy, accumulator


inits=dict(solution=np.random.uniform(0,1,5), length=100, T=50000)
bmeta, bs, be, out = sa(f3, inits, 30, tf, itf, pf)

Epoch 0
Temperature 50000 Length 100
Epoch 1
Temperature 40000.0 Length 120
Epoch 2
Temperature 32000.0 Length 144
Epoch 3
Temperature 25600.0 Length 173
Epoch 4
Temperature 20480.0 Length 208
Epoch 5
Temperature 16384.0 Length 250
Epoch 6
Temperature 13107.2 Length 300
Epoch 7
Temperature 10485.760000000002 Length 360
Epoch 8
Temperature 8388.608000000002 Length 432
Epoch 9
Temperature 6710.886400000002 Length 519
Epoch 10
Temperature 5368.709120000002 Length 623
Epoch 11
Temperature 4294.967296000002 Length 748
Epoch 12
Temperature 3435.9738368000017 Length 898
Epoch 13
Temperature 2748.7790694400014 Length 1078
Epoch 14
Temperature 2199.023255552001 Length 1294
Epoch 15
Temperature 1759.218604441601 Length 1553
Epoch 16
Temperature 1407.3748835532808 Length 1864
Epoch 17
Temperature 1125.8999068426247 Length 2237
Epoch 18
Temperature 900.7199254740998 Length 2685
Epoch 19
Temperature 720.5759403792799 Length 3222
Epoch 20
Temperature 576.460752303424 Length 3867
Epoch 21
Temperature

In [114]:
be

-7.656308091595568

In [118]:
bs

array([-0.04702748,  0.10920125, -0.094156  , -0.16453894, -0.24823388])

In [92]:
h= map(f3, range(0, 100))